In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import gzip
import json
import pandas as pd
from pymongo import MongoClient
from sklearn.metrics import roc_auc_score
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f7cd5cb41f0>>
Traceback (most recent call last):
  File "/home/dhruvahuja19/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


Parse json files and store them in pandas df

In [ ]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)
def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')
purchasing_df = getDF('../../../Data/Video_Games_5.json.gz')
meta_datadf = getDF('../../../Data/meta_Video_Games.json.gz')

FileNotFoundError: [Errno 2] No such file or directory: '../../../Data/Video_Games_5.json.gz'

Dropping irrelevant columns in positive samples, labeling all purchasing data as a positive sample, and coverting reviewTime to a datatime obj

In [4]:
columns_to_drop = ['vote', 'style', 'image'] 
purchasing_df = purchasing_df.drop(columns=columns_to_drop)
positive_samples = purchasing_df.copy()
positive_samples['label'] = 1 
positive_samples['reviewTime'] = pd.to_datetime(positive_samples['reviewTime'])
positive_samples.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,label
0,5.0,True,2015-10-17,A1HP7NVNPFMA4N,0700026657,Ambrosia075,"This game is a bit hard to get the hang of, bu...",but when you do it's great.,1445040000,1
1,4.0,False,2015-07-27,A1JGAP0185YJI6,0700026657,travis,I played it a while but it was alright. The st...,"But in spite of that it was fun, I liked it",1437955200,1
2,3.0,True,2015-02-23,A1YJWEXHQBWK2B,0700026657,Vincent G. Mezera,ok game.,Three Stars,1424649600,1
3,2.0,True,2015-02-20,A2204E1TH211HT,0700026657,Grandma KR,"found the game a bit too complicated, not what...",Two Stars,1424390400,1
4,5.0,True,2014-12-25,A2RF5B5H74JLPE,0700026657,jon,"great game, I love it and have played it since...",love this game,1419465600,1


Converting the list of also_view items to a set to speed up runtime later. Dropping duplicates in the metadata because otherwise when we inner join with our positive samples, we get a nasty cross product that duplicates some of the positive samples unnecesisarily. Reset the index after dropping the duplicates

In [5]:
def process_asin_list(asin_list):
    return set(asin_list)

meta_datadf['also_view'] = meta_datadf['also_view'].apply(process_asin_list)
meta_datadf = meta_datadf.reset_index()
meta_datadf = meta_datadf.drop_duplicates(subset='asin', keep='first')
meta_datadf.reset_index(drop=True, inplace=True)
meta_datadf.head()

,index,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,0,"[Video Games, PC, Games]",,[],,Reversi Sensory Challenger,[],,Fidelity Electronics,[],"[>#2,623,937 in Toys &amp; Games (See Top 100 ...",{},Toys &amp; Games,,,,0042000742,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
1,1,"[Video Games, Xbox 360, Games, </span></span><...",,[Brand new sealed!],,Medal of Honor: Warfighter - Includes Battlefi...,[B00PADROYW],,by\n \n EA Games,[],"[>#67,231 in Video Games (See Top 100 in Video...","{B072NQJCW5, B0050SY5BM, B000TI836G, B000TG530...",Video Games,,,"\n\t\t\t\t\t\t\t\t\t\t\t\t<span class=""vertica...",0078764343,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
2,2,"[Video Games, Retro Gaming & Microconsoles, Su...",,[],,street fighter 2 II turbo super nintendo snes ...,[],,Nintendo,[],"[>#134,433 in Video Games (See Top 100 in Vide...",{},Video Games,,,$0.72,0276425316,[],[],NaN
3,3,"[Video Games, Xbox 360, Accessories, Controlle...",,[MAS's Pro Xbox 360 Stick (Perfect 360 Stick) ...,,Xbox 360 MAS STICK,[],,by\n \n MAS SYSTEMS,[Original PCB used from Xbox 360 Control Pad (...,"[>#105,263 in Video Games (See Top 100 in Vide...",{},Video Games,,,,0324411812,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
4,4,"[Video Games, PC, Games, </span></span></span>...",,"[Phonics Alive! 3, The Speller teaches student...",,Phonics Alive! 3: The Speller,[],,by\n \n Advanced Software Pty. Ltd.,"[Grades 2-12, Spelling Program, Teaches Spelli...","[>#92,397 in Video Games (See Top 100 in Video...",{B000BCZ7U0},Video Games,,,,0439335310,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN


Merging the metadata and the positive samples, giving us access to the also_view category. Now we have [asin, price, also_view]. Convert string prices to floats, otherwise place an empty string. We dont drop the NANs, as we fill them in later

In [6]:
positive_samples = positive_samples.merge(meta_datadf, on='asin', how='inner')
positive_samples['price'] = pd.to_numeric(positive_samples['price'].str.replace(r'[^\d.]', '', regex=True), errors='coerce')
positive_samples['price'].isna().sum()

428894

In [7]:
users_to_instances = {} 
for index, row in positive_samples.iterrows(): 
    if users_to_instances.get(row['reviewerID']) is None:
        users_to_instances[row['reviewerID']] = 0
    users_to_instances[row['reviewerID']] += 1

In [8]:
#create a dataset containing only rows where users_to_instance[row[reviewerID]] >= 10 
positive_samples = positive_samples[positive_samples['reviewerID'].map(users_to_instances) >= 10]

In [9]:
item_to_instances = {} 
for index, row in positive_samples.iterrows(): 
    if item_to_instances.get(row['asin']) is None:
        item_to_instances[row['asin']] = 0
    item_to_instances[row['asin']] += 1

In [10]:
positive_samples = positive_samples[positive_samples['asin'].map(item_to_instances) >= 10]

In [11]:
positive_samples.shape

(186198, 29)

More irrelevant columns; can probably merged with above cell

In [12]:
columns_to_drop = ['imageURL', 'imageURLHighRes', 'reviewerName', 'reviewText', 
                   'unixReviewTime', 'date']
positive_samples = positive_samples.drop(columns=columns_to_drop, axis=1)
positive_samples = positive_samples.reset_index(drop=True)

In [13]:
negative_samples = pd.DataFrame(columns=positive_samples.columns)

Load MongoDB data,project out the asin, and corresponding dictionary that maps times to prices 

In [14]:
client = MongoClient('mongodb://localhost:27017/')
db = client['video_games_db']
price_collection = db['keepa']
query = {"df_NEW": {"$exists": True}}
document_count = price_collection.count_documents(query)
projection = {"df_NEW": 1, "asin": 1, "_id": 0}
documents = price_collection.find(query, projection)
price_df = pd.DataFrame(documents) 

To avoid out of bounds erorrs: If there exists an asin in the purchasing data that doesnt exist in the metadata, then we don't want to search the metadata for it. In hindsight, this might be unncessary given that the inner join probably would have deleted the row from the corresponding table

In [15]:
items_considered = set() 
for index, row in positive_samples.iterrows(): 
    items_considered.add(row["asin"])
items_considered - set(meta_datadf["asin"])

set()

The next three cells are just for runtime purposes; We treat asin as a primary key. We don't want to have to have to exhaustive search df["asin" == stuff] to find a row. We essentially treat asin as a primary key, although there may be duplicate keys, however we just pick the last one WLOG as we replace the time-variant/person-variant features

In [16]:
asin_to_price = {}
for index, row in price_df.iterrows():
    asin_to_price[row["asin"]] = row["df_NEW"]

In [17]:
asin_to_positive = {} 
for index, row in positive_samples.iterrows(): 
    asin_to_positive[row['asin']] = index

In [18]:
asin_to_meta = {}
for index, row in meta_datadf.iterrows(): 
    asin_to_meta[row["asin"]] = index

creating negative samples. The price doesn't matter for now

In [19]:
from tqdm import tqdm
tqdm.pandas()
dfs_to_concat = [] 
for index, row in tqdm(positive_samples.iterrows(), total=positive_samples.shape[0], desc="Processing rows"):
    current_reviewer = row['reviewerID'] 
    also_view_items = meta_datadf.iloc[asin_to_meta[row['asin']]]['also_view']
    for item in also_view_items: 
        if item in items_considered: 
            current_asin = item 
            current_date = pd.Timestamp(row['reviewTime'])
            new_row = positive_samples.iloc[asin_to_positive[current_asin]].to_dict()
            new_row['label'] = 0
            new_row['reviewTime'] = current_date
            new_row["reviewerID"] = current_reviewer
            dfs_to_concat.append(new_row)

Processing rows: 100%|██████████| 186198/186198 [03:09<00:00, 983.92it/s] 


In [20]:
negative_samples = pd.DataFrame(dfs_to_concat)
print(negative_samples.shape)
print(positive_samples.shape)

(4150451, 23)
(186198, 23)


In [21]:
combined_samples = pd.concat([positive_samples, negative_samples], ignore_index=True)


In [22]:
print(combined_samples.head())

   overall  verified reviewTime      reviewerID        asin  \
0      5.0      True 2011-08-14   AN3YYDZAS3O1Y  0700099867   
1      5.0     False 2011-06-18  A15PIAQT55GNCA  0700099867   
2      4.0      True 2011-06-14  A361M14PU2GUEG  0700099867   
3      5.0     False 2011-06-13  A2LQCBLLJVVR5T  0700099867   
4      1.0      True 2014-06-30  A248LSBZT4P38V  0700099867   

                                             summary  label  index  \
0        A step up from Dirt 2 and that is terrific!      1     41   
1                      this games is amazing!!!!!!!!      1     41   
2                                             DIRT 3      1     41   
3              BEST GRAPHICS OF ANY GAME SO FAR !!!!      1     41   
4  It might have been a good game, but I never fo...      1     41   

                   category tech1  ... also_buy tech2   brand  \
0  [Video Games, PC, Games]        ...       []        Dirt 3   
1  [Video Games, PC, Games]        ...       []        Dirt 3   
2  [V

In [23]:
def binary_search_closest(arr, target, compare):
    low, high = 0, len(arr) - 1
    closest = None
    closest_diff = float('inf')
    while low <= high:
        mid = (low + high) // 2
        result = compare(arr[mid], target)
        if result == 0:
            return arr[mid]
        elif result < 0:
            low = mid + 1
        else:
            high = mid - 1
        diff = abs(result)
        if diff < closest_diff:
            closest_diff = diff
            closest = arr[mid]
    return closest

Update price information using MongoDB data

In [24]:
from tqdm import tqdm
tqdm.pandas()
removed_rows = set()
for index, row in tqdm(combined_samples.iterrows(), total=combined_samples.shape[0], desc="Processing rows"):
    asin = row["asin"]
    price = row["price"]
    current_date = row["reviewTime"]
    if asin in asin_to_price:
        price_list = list(asin_to_price[asin].keys())
        closest_date = binary_search_closest(price_list, current_date, lambda x, y: (pd.to_datetime(x) - y).total_seconds())
        closest_price = asin_to_price[asin][closest_date]
        combined_samples.at[index, "price"] = closest_price
        # if np.isnan(closest_price):
        #         print("here")
    else:
        removed_rows.add(index)
print(len(removed_rows))

Processing rows: 100%|██████████| 4336649/4336649 [1:36:54<00:00, 745.85it/s]  

261748


Remove rows not present in the mongoDB data

In [25]:
combined_samples = combined_samples.drop(removed_rows)
combined_samples.reset_index(drop=True, inplace=True)

Remove rows that have nan prices in the mongoDB data

In [26]:
combined_samples = combined_samples.dropna(subset=['price'])
combined_samples.reset_index(drop=True, inplace=True)

In [5]:
#shuffle combined_samples 
combined_samples = combined_samples.sample(frac=1).reset_index(drop=True)

Label encode categorical variables

In [6]:
print(len(combined_samples))

3609211


In [8]:
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()
brand_encoder = LabelEncoder()
combined_samples = combined_samples[['user_id_encoded', 'item_id_encoded', 'brand_id_encoded', 'price', 'description', 'label']]
num_users =  combined_samples["user_id_encoded"].nunique()
num_items = combined_samples["item_id_encoded"].nunique()
num_brands =  combined_samples["brand_id_encoded"].nunique()

In [9]:
X = combined_samples[['user_id_encoded', 'item_id_encoded', 'brand_id_encoded', 'price', 'description']].values
y = combined_samples['label'].values

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

positive_indices = np.where(y_train == 1)[0]
X_train_positive = X_train[positive_indices]
y_train_positive = y_train[positive_indices]
positive_indices_test = np.where(y_test == 1)[0]
X_test_positive = X_test[positive_indices_test]
y_test_positive = y_test[positive_indices_test]
for i in range(20):
    X_train = np.concatenate((X_train, X_train_positive))
    y_train = np.concatenate((y_train, y_train_positive))
    X_test = np.concatenate((X_test, X_test_positive))
    y_test = np.concatenate((y_test, y_test_positive))

permutation = np.random.permutation(len(X_train))
X_train_shuffled = X_train[permutation]
y_train_shuffled = y_train[permutation]

X_train_user = X_train[:, 0].astype(np.int32)
X_train_item = X_train[:,1].astype(np.int32)
X_train_brand = X_train[:,2].astype(np.int32)
X_train_price = X_train[:,3].astype(np.float32)
X_train_description = X_train[:, 4].astype(np.str_)

X_test_user = X_test[:, 0].astype(np.int32)
X_test_item = X_test[:, 1].astype(np.int32)
X_test_brand = X_test[:, 2].astype(np.int32)
X_test_price = X_test[:, 3].astype(np.float32)
X_test_description = X_test[:, 4].astype(np.str_)

y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

In [ ]:
tokenizer = get_tokenizer("basic_english")
def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)
vocab = build_vocab_from_iterator(yield_tokens(X_train_description), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])
def text_pipeline(text):
    return [vocab[token] for token in tokenizer(text)]

X_train_description_processed = [text_pipeline(text) for text in X_train_description]
X_test_description_processed = [text_pipeline(text) for text in X_test_description]
max_length = max(len(desc) for desc in X_train_description_processed + X_test_description_processed)

def pad_sequence(seq):
    return seq + [vocab["<unk>"]] * (max_length - len(seq))

X_train_description_padded = torch.tensor([pad_sequence(desc) for desc in X_train_description_processed], dtype=torch.long)
X_test_description_padded = torch.tensor([pad_sequence(desc) for desc in X_test_description_processed], dtype=torch.long)

In [ ]:
X_train_user = torch.tensor(X_train_user, dtype=torch.int32)
X_train_item = torch.tensor(X_train_item, dtype=torch.int32)
X_train_brand = torch.tensor(X_train_brand, dtype=torch.int32)
X_train_price = torch.tensor(X_train_price, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)

X_test_user = torch.tensor(X_test_user, dtype=torch.int32)
X_test_item = torch.tensor(X_test_item, dtype=torch.int32)
X_test_brand = torch.tensor(X_test_brand, dtype=torch.int32)
X_test_price = torch.tensor(X_test_price, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

train_dataset = TensorDataset(X_train_user, X_train_item, X_train_brand, X_train_price, X_train_description_padded, y_train)
test_dataset = TensorDataset(X_test_user, X_test_item, X_test_brand, X_test_price, X_test_description_padded, y_test)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

class RecommendationModel(nn.Module):
    def __init__(self, num_users, num_items, num_brands, vocab_size, embedding_size, description_embedding_size):
        super(RecommendationModel, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_size)
        self.item_embedding = nn.Embedding(num_items, embedding_size)
        self.brand_embedding = nn.Embedding(num_brands, embedding_size)
        self.description_embedding = nn.Embedding(vocab_size, description_embedding_size)
        
        self.fc1 = nn.Linear(embedding_size * 3 + 1 + description_embedding_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, user, item, brand, price, description):
        user_vec = self.user_embedding(user)
        item_vec = self.item_embedding(item)
        brand_vec = self.brand_embedding(brand)
        desc_vec = self.description_embedding(description).mean(dim=1)  # Average pooling
        
        x = torch.cat([user_vec, item_vec, brand_vec, price.unsqueeze(1), desc_vec], dim=1)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, p=0.5)
        x = torch.sigmoid(self.fc3(x))
        return x


In [ ]:
vocab_size = len(vocab)
model = RecommendationModel(num_users=num_users, num_items=num_items, num_brands=num_brands, vocab_size=vocab_size, embedding_size=32, description_embedding_size=100)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
def calculate_accuracy(outputs, labels):
    predictions = (torch.sigmoid(outputs) > 0.5).float()
    correct = (predictions == labels).float().sum()
    return correct / len(labels)

for epoch in range(50):
    model.train()
    all_outputs = []
    all_labels = []
    total_loss = 0
    total_accuracy = 0
    num_batches = 0
    for users, items, brands, prices, descriptions, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(users, items, brands, prices, descriptions)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        total_accuracy += calculate_accuracy(outputs.squeeze(), labels)
        num_batches += 1
        all_outputs.extend(outputs.detach().cpu().numpy())
        all_labels.extend(labels.detach().cpu().numpy())

    train_auc = roc_auc_score(all_labels, all_outputs)
    train_loss = total_loss / num_batches
    train_accuracy = total_accuracy / num_batches
    
    model.eval()
    total_val_loss = 0
    total_val_accuracy = 0
    num_val_batches = 0
    all_val_outputs = []
    all_val_labels = []
    
    with torch.no_grad():
        for users, items, brands, prices, descriptions, labels in test_loader:
            outputs = model(users, items, brands, prices, descriptions)
            val_loss = criterion(outputs.squeeze(), labels)
            total_val_loss += val_loss.item()
            total_val_accuracy += calculate_accuracy(outputs.squeeze(), labels)
            num_val_batches += 1
            all_val_outputs.extend(outputs.detach().cpu().numpy())
            all_val_labels.extend(labels.detach().cpu().numpy())
        
    val_loss = total_val_loss / num_val_batches
    val_accuracy = total_val_accuracy / num_val_batches
    model.train()
    val_auc = roc_auc_score(all_val_labels, all_val_outputs)
    print(f"Epoch {epoch+1}/{50}")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
    print(f"Train AUC: {train_auc:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")
    print(f"Val AUC: {val_auc:.4f}")
    print("--------------------")